In [3]:
import pandas as pd
import time
import random

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from IPython.display import display
from datetime import datetime


### Mercadona Web Scraping Script

In [48]:
# 1. Abre navegador en blanco
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# 2. Abrimos web mercadona
url_mercadona ="https://tienda.mercadona.es/categories/"
driver.get(url_mercadona)

# Esperamos unos segundillos
time.sleep(3)

# Aceptamos cookies
cookies = driver.find_element(By.CLASS_NAME,"ui-button.ui-button--small.ui-button--primary.ui-button--positive")
cookies.send_keys(Keys.ENTER)

# Buscamos por clase el campo de input para poder introducir nuestro código postal
input = driver.find_element(By.CLASS_NAME,"ym-hide-content")

# Introducimos el código postal
input.send_keys("28016")

# Esperamos unos segundillos
time.sleep(round(random.uniform(0.5,3),1))

# Le damos a ENTER
input.send_keys(Keys.ENTER)   #driver.find_element_by_id("submit").click()


[WDM] - Downloading: 100%|██████████| 8.82M/8.82M [00:01<00:00, 9.13MB/s]


In [ ]:
driver.close()

In [49]:
# dict_mercadona = {"name":[],"size":[],"price":[], "url":[],"unit_cost":[],"units":[]}
dict_mercadona = {"Group":[],"Category":[],"Name":[],"Size":[],"Price":[], "Image":[]}

categories_length = len(driver.find_elements(By.CLASS_NAME,"category-menu__item"))+1
categories_range = range(1,categories_length)

categories = [] # Títulos de las categorías
sub_header_headers= [] # Títulos de los sub-headers
sub_category= []
empty =[]

category_contador = 0
group_contador = 0
sub_category_contador = 0

def get_product_details(categories,empty,group_contador,category_contador):
    # ONCE IN PRODUCT DISPLAY => RECORRER CADA PRODUCTO Y PINCHAR EN ÉL
    contenedor = driver.find_element(By.CLASS_NAME,"category-detail__content")
    contenedores_grupo = contenedor.find_elements(By.CLASS_NAME,"product-container")

    list_products = []
    list_empty = []
    
    for contenedor in contenedores_grupo:
        
        product_container = contenedor.find_elements(By.CLASS_NAME,"product-cell")
        
        for product in product_container:
            product_image = contenedor.find_element(By.TAG_NAME,"img").get_attribute("src")
            product_array = product.text.split("\n")[:-1]
            product_array.append(product_image)
            
            # ADD ALL PRODUCT INFO TO EMPTY ARRAY
            dict_mercadona["Name"].append(product_array[0])
            dict_mercadona["Size"].append(product_array[1])
            dict_mercadona["Price"].append(product_array[2])
            dict_mercadona["Image"].append(product_array[-1])
            dict_mercadona["Group"].append(categories[group_contador])
            dict_mercadona["Category"].append(empty[category_contador])
            

            list_empty.append(product_array)

        list_products.append(list_empty)
        list_empty = []


for num in categories_range:
    
    # Nivel 1 => GROUP:
    header_a_pinchar = driver.find_elements(By.CSS_SELECTOR,f"#root > div.grid-layout > div.grid-layout__sidebar > ul > li:nth-child({num}) > div > button")    
    categories.append(header_a_pinchar[0].text)
    header_a_pinchar[0].click()
    time.sleep(1)

    # Nivel 2 => CATEGORÍAS:
    sub_category_length = len(driver.find_elements(By.CSS_SELECTOR,"#root > div.grid-layout > div.grid-layout__sidebar > ul > li.category-menu__item.open > div > ul")[0].text.split("\n"))+1
    number_of_sub_headers = range(1,sub_category_length)


    # Pincha todos los botones sub headers y almacena los títulos
    for i in number_of_sub_headers:

        sub_header_button = driver.find_elements(By.CSS_SELECTOR,f"#root > div.grid-layout > div.grid-layout__sidebar > ul > li.category-menu__item.open > div > ul > li:nth-child({i}) > button")        
        empty.append(sub_header_button[0].text)
        sub_header_button[0].click()
        time.sleep(1)
        # Una vez en el bloque adecuado cogemos todos los productos
        get_product_details(categories,empty,group_contador,category_contador)
        category_contador+=1
        sub_category_contador+=1

    group_contador+=1
    
    # sub_header_headers.append(empty)
    # empty =[]
     


df_mercadona = pd.DataFrame(dict_mercadona)


KeyboardInterrupt: 

In [47]:
prices = df_mercadona["Price"]
formated_prices = []

for price in prices:

    if len(price)<5:
        print(price)
        # formated_prices.append(float(price.split(" ")[0].replace(",",".")))
    else:
        # formated_prices.append("NaN")

    

# print(formated_prices)


IndentationError: expected an indented block (2338027167.py, line 13)

In [18]:
df_mercadona['Shop'] = 'Mercadona'

today = pd.Timestamp.now()
df_mercadona['Date'] = today

In [19]:
df_mercadona

,Group,Category,Name,Size,Price,Image,Shop,Date
0,"Aceite, especias y salsas","Aceite, vinagre y sal","Aceite de oliva 0,4º Hacendado",Garrafa 5 L,"23,63 € /ud.",https://prod-mercadona.imgix.net/images/e4b36f...,Mercadona,2023-03-05 23:49:51.711541
1,"Aceite, especias y salsas","Aceite, vinagre y sal","Aceite de oliva 0,4º Hacendado",Botella 1 L,"4,77 € /ud.",https://prod-mercadona.imgix.net/images/e4b36f...,Mercadona,2023-03-05 23:49:51.711541
2,"Aceite, especias y salsas","Aceite, vinagre y sal",Aceite de oliva virgen extra Hacendado,Garrafa 3 L,"16,75 € /ud.",https://prod-mercadona.imgix.net/images/e4b36f...,Mercadona,2023-03-05 23:49:51.711541
3,"Aceite, especias y salsas","Aceite, vinagre y sal",Aceite de oliva virgen extra Hacendado,Botella 1 L,"5,63 € /ud.",https://prod-mercadona.imgix.net/images/e4b36f...,Mercadona,2023-03-05 23:49:51.711541
4,"Aceite, especias y salsas","Aceite, vinagre y sal",Aceite de oliva virgen extra Hacendado Gran Se...,Botella 750 ml,"5,11 € /ud.",https://prod-mercadona.imgix.net/images/e4b36f...,Mercadona,2023-03-05 23:49:51.711541
...,...,...,...,...,...,...,...,...
5357,Zumos,Tomate y otros sabores,Bebida de pomelo Hacendado sin azúcares añadidos,Brick 1 L,"1,25 € /ud.",https://prod-mercadona.imgix.net/images/ec1076...,Mercadona,2023-03-05 23:49:51.711541
5358,Zumos,Tomate y otros sabores,Bebida guayaba Hacendado sin azúcares añadidos,Brick 1 L,"1,30 € /ud.",https://prod-mercadona.imgix.net/images/ec1076...,Mercadona,2023-03-05 23:49:51.711541
5359,Zumos,Tomate y otros sabores,Bebida de pera Hacendado sin azúcares añadidos,Brick 1 L,"1,10 € /ud.",https://prod-mercadona.imgix.net/images/ec1076...,Mercadona,2023-03-05 23:49:51.711541
5360,Zumos,Tomate y otros sabores,Zumo de pomelo Hacendado,Brick 1 L,"1,45 € /ud.",https://prod-mercadona.imgix.net/images/ec1076...,Mercadona,2023-03-05 23:49:51.711541


#### CREATE CSV + SAVE AS EXCEL

In [20]:
df_mercadona.to_csv("./data/mercadona.csv",index=False)

read_file = pd.read_csv ("./data/mercadona.csv")
read_file.to_excel ("./data/mercadona.xlsx", index = None, header=True)

#### APPEND TO EXISTING FILE

In [57]:
df_mercadona.to_csv("./data/mercadona.csv", mode='a', index=False, header=False)

read_file = pd.read_csv ("./data/mercadona.csv")
read_file.to_excel ("./data/mercadona.xlsx", index = None, header=True)


In [12]:
mercadona = pd.read_csv("./data/mercadona.csv")
mercadona

,Group,Category,Name,Size,Price,Image,Shop,Date
0,"Aceite, especias y salsas,""Aceite, vinagre y s...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Aceite, especias y salsas,""Aceite, vinagre y s...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Aceite, especias y salsas,""Aceite, vinagre y s...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Aceite, especias y salsas,""Aceite, vinagre y s...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Aceite, especias y salsas,""Aceite, vinagre y s...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
5360,"Zumos,Tomate y otros sabores,Bebida de pomelo ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
5361,"Zumos,Tomate y otros sabores,Bebida guayaba Ha...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
5362,"Zumos,Tomate y otros sabores,Bebida de pera Ha...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
5363,"Zumos,Tomate y otros sabores,Zumo de pomelo Ha...",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
price = "23,63 € /ud."

float(price.split(" ")[0].replace(",","."))

23.63

In [11]:
mercadona["Price"]= mercadona["Price"].apply(lambda x:float(x.split(" ")[0].replace(",",".")))
mercadona["Price"]

ValueError: could not convert string to float: 'https://prod-mercadona.imgix.net/images/0193fc9cf60e3a8661fda34775409fce.jpg?fit=crop&h=300&w=300'

### Format date so that pandas can understand them

In [667]:
mercadona['Date'] = pd.to_datetime(mercadona['Date'])

In [668]:
mercadona.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1266 entries, 0 to 1265
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Group     1266 non-null   object        
 1   Category  1266 non-null   object        
 2   Name      1266 non-null   object        
 3   Size      1266 non-null   object        
 4   Price     1266 non-null   object        
 5   Image     1266 non-null   object        
 6   Shop      1266 non-null   object        
 7   Date      1266 non-null   datetime64[ns]
dtypes: datetime64[ns](1), object(7)
memory usage: 79.2+ KB


In [366]:
categories

['Aceite, especias y salsas', 'Agua y refrescos']

In [367]:
sub_header_headers

[['Aceite, vinagre y sal',
  'Especias',
  'Mayonesa, ketchup y mostaza',
  'Otras salsas'],
 ['Agua',
  'Isotónico y energético',
  'Refresco de cola',
  'Refresco de naranja y de limón',
  'Tónica y bitter',
  'Refresco de té y sin gas']]

In [42]:
category_container = driver.find_element(By.CLASS_NAME,"category-detail__content")
longitud_contenedores_pequeños = len(category_container.find_elements(By.CLASS_NAME,"section"))
h2_contenedor = category_container.find_elements(By.TAG_NAME,"h2")

for i in h2_contenedor:
    print(i.text)

Fregonas, escobas y mopas
Cubos y barreños
Otros utensilios de limpieza
Limpieza de calzado


In [50]:

# ONCE IN PRODUCT DISPLAY => RECORRER CADA PRODUCTO Y PINCHAR EN ÉL
contenedor = driver.find_element(By.CLASS_NAME,"category-detail__content")
contenedores_grupo = contenedor.find_elements(By.CLASS_NAME,"product-container")

list_products = []
list_empty = []

for contenedor in contenedores_grupo:
    product_container = contenedor.find_elements(By.CLASS_NAME,"product-cell")
        
    for product in product_container:
        
        product_image = contenedor.find_element(By.TAG_NAME,"img").get_attribute("src")
        product_array = product.text.split("\n")[:-1]
        product_array.append(product_image)
        
    
        # # OPEN PRODUCT
        # open_product_buton = product.find_element(By.CLASS_NAME,"product-cell__content-link")
        # open_product_buton.send_keys(Keys.ENTER)
        # time.sleep(1)

        # # ACCESS PRICE/KG/L
        # clicked_product_container = driver.find_element(By.CLASS_NAME,"modal-content")
        # get_price_per_unit = clicked_product_container.find_elements(By.CLASS_NAME,"headline1-r")[-1]

        # # Format price:
        # get_price_per_unit = get_price_per_unit.text.split(" ")
        # price_per_unit = float(get_price_per_unit[1].replace(",","."))
        # measure_per_unit = get_price_per_unit[2]

        # # ADD PRICES AND UNITS
        # product_array.append(price_per_unit)
        # product_array.append(measure_per_unit)

        # # CLOSE PRODUCT PREVIEW
        # close_product_button = driver.find_element(By.CLASS_NAME,"modal-content__close") 
        # close_product_button.send_keys(Keys.ENTER)
        # time.sleep(0.5)
        
        # ADD ALL PRODUCT INFO TO EMPTY ARRAY
        list_empty.append(product_array)
        

    list_products.append(list_empty)
    
    list_empty = []
    

print("\n----------------------\n")
print(list_products)


----------------------

[[['Fregona Microfibra Absorbente Bosque Verde', 'Paquete 2 ud.', '2,95 € /ud.', 'https://prod-mercadona.imgix.net/images/0193fc9cf60e3a8661fda34775409fce.jpg?fit=crop&h=300&w=300'], ['Fregona Microfibra Absorbente Bosque Verde', 'Paquete 1 ud.', '1,50 € /ud.', 'https://prod-mercadona.imgix.net/images/0193fc9cf60e3a8661fda34775409fce.jpg?fit=crop&h=300&w=300'], ['Fregona Algodón Bosque Verde', 'Paquete 1 ud.', '1,10 € /ud.', 'https://prod-mercadona.imgix.net/images/0193fc9cf60e3a8661fda34775409fce.jpg?fit=crop&h=300&w=300'], ['Fregona Resistente y Absorbente Bosque Verde', 'Paquete 2 ud.', '1,85 € /ud.', 'https://prod-mercadona.imgix.net/images/0193fc9cf60e3a8661fda34775409fce.jpg?fit=crop&h=300&w=300'], ['Fregona Suave Bosque Verde', 'Paquete 1 ud.', '0,80 € /ud.', 'https://prod-mercadona.imgix.net/images/0193fc9cf60e3a8661fda34775409fce.jpg?fit=crop&h=300&w=300'], ['Fregona Microfibra Resistente Bosque Verde', 'Paquete 1 ud.', '3,80 € /ud.', 'https://prod-mer

In [544]:
price_to_format = "| 2,667 €/L"
price_to_format = price_to_format.split(" ")
price_per_unit = float(price_to_format[1].replace(",","."))
measure_per_unit = price_to_format[2]



2.667

In [434]:

contenedor = driver.find_element(By.CLASS_NAME,"category-detail__content")
longitud_contenedores_pequeños = len(contenedor.find_elements(By.CLASS_NAME,"section"))
h2_contenedor = contenedor.find_elements(By.TAG_NAME,"h2")
contenedores_grupo = contenedor.find_elements(By.CLASS_NAME,"product-container")
# contenedor_producto = contenedores_grupo.find_element(By.CLASS_NAME,"product-cell")

#root > div.grid-layout > div.grid-layout__main-container > div.grid-layout__content > div > div > section:nth-child(2) > div > div:nth-child(1)
#root > div.grid-layout > div.grid-layout__main-container > div.grid-layout__content > div > div > section:nth-child(2) > div > div:nth-child(2)

# for i in h2_contenedor:
#     print(i.text)

#print(contenedor.find_elements(By.CLASS_NAME,"section")[1].text)
# print(contenedores_pequeños[0].text.split("\n")[1:])

for i in contenedores_grupo:
    chechu = i.find_elements(By.CLASS_NAME,"product-cell")

    for x in chechu:
        print(x.text.split("\n")[:-1])



    # print(i.find_elements(By.CLASS_NAME,"product-cell")[1].text)
    # print(i.text.split("\n"))
    print("_________________________\n")

['Refresco al limón Nestea', '8 latas x 330 ml', '6,96 € /pack']
['Refresco al limón Nestea', 'Lata 330 ml', '0,90 € /ud.']
['Refresco al limón Nestea', 'Botella 1,5 L', '1,95 € /ud.']
['Refresco té verde Hacendado', 'Botella 1,5 L', '1,05 € /ud.']
['Refresco té sabor limón Hacendado cero', 'Botella 1,5 L', '0,95 € /ud.']
['Refresco té sabor mango Hacendado cero', 'Botella 1,5 L', '1,05 € /ud.']
['Refresco té sabor limón Hacendado', 'Botella 1,5 L', '0,95 € /ud.']
['Refresco té sabor melocotón Hacendado cero', 'Botella 1,5 L', '0,95 € /ud.']
['Refresco té con limón Hacendado para diluir', 'Caja 12 sobres (36 g)', '1,30 € /ud.']
['Refresco té con melocotón Hacendado para diluir', 'Caja 12 sobres (36 g)', '1,30 € /ud.']
_________________________

['Limonada Don Simón light sin gas', 'Botella 2 L', '1,50 € /ud.']
['Refresco de naranja Simon Life sin gas', 'Botella 1,5 L', '1,85 € /ud.']
['Refresco de naranja Gold Spring sin gas', 'Botella 2 L', '1,30 € /ud.']
['Refresco de naranja Gold Sp

In [362]:

# Coger todos los h2
container_section_h2 = driver.find_elements(By.CSS_SELECTOR,"#root > div.grid-layout > div.grid-layout__main-container > div.grid-layout__content > div > div > section > h2")
container_section_div = driver.find_elements(By.CSS_SELECTOR,"#root > div.grid-layout > div.grid-layout__main-container > div.grid-layout__content > div > div > section > div")

for i in container_section_h2:
    print(i.text)


# for j in container_section_div:
#     print(j.text.split("\n")[:-1])

        


# Recorre todos los productos de la página
# container_section = driver.find_elements(By.CSS_SELECTOR,"#root > div.grid-layout > div.grid-layout__main-container > div.grid-layout__content > div > div > section > div > div")
# for i in container_section:
#     print(i.text.split("\n")[:-1])



Aceite de oliva
Aceite de girasol, semillas y maíz
Vinagre y otros aderezos
Sal y bicarbonato


In [162]:
# Iteramos sobre el menu:
menu_alimentos = driver.find_elements(By.CLASS_NAME,"category-menu__header")

categorias = []

# Cogemos todas las categorías que tiene la página web.
# Si mañana se modifica/añade/elimina categoría queremos que siempre se pueda recorrer
for menu in menu_alimentos:
    categorias.append(menu.text)
    botones = driver.find_element(By.TAG_NAME, "button")

print(categorias)
print("Tenemos",len(categorias),"categorias")

['Aceite, especias y salsas', 'Agua y refrescos', 'Aperitivos', 'Arroz, legumbres y pasta', 'Azúcar, caramelos y chocolate', 'Bebé', 'Bodega', 'Cacao, café e infusiones', 'Carne', 'Cereales y galletas', 'Charcutería y quesos', 'Congelados', 'Conservas, caldos y cremas', 'Cuidado del cabello', 'Cuidado facial y corporal', 'Fitoterapia y parafarmacia', 'Fruta y verdura', 'Huevos, leche y mantequilla', 'Limpieza y hogar', 'Maquillaje', 'Marisco y pescado', 'Mascotas', 'Panadería y pastelería', 'Pizzas y platos preparados', 'Postres y yogures', 'Zumos']
Tenemos 26 categorias


In [214]:
headers = driver.find_elements(By.CLASS_NAME,"category-menu__item")

print(len(headers))

26


In [261]:
# Pincha sobre todos los headers de categorías

#root > div.grid-layout > divgrid-layout__sidebar > ul > li:nth-child(26)

# Total categories length:
total_categories_length = len(driver.find_elements(By.CLASS_NAME,"category-menu__item"))
numero_headers = range(1,3)

categorias = []
sub_categorias = []


for num in numero_headers:
    
    header_a_pinchar = driver.find_elements(By.CSS_SELECTOR,f"#root > div.grid-layout > div.grid-layout__sidebar > ul > li:nth-child({num}) > div > button")

    # CATEGORÍAS:
    categorias.append(header_a_pinchar[0].text)
    header_a_pinchar[0].click()

    # SUB_HEADERS:
    selection = driver.find_elements(By.CSS_SELECTOR,"#root > div.grid-layout > div.grid-layout__sidebar > ul > li.category-menu__item.open > div > ul")

    selection[0].click()
    time.sleep(round(random.uniform(0.5,3),1))
    sub_categorias.append(selection[0].text.split('\n'))


    time.sleep(round(random.uniform(0.5,3),1))

print(sub_categorias)

[['Aceite, vinagre y sal', 'Especias', 'Mayonesa, ketchup y mostaza', 'Otras salsas'], ['Agua', 'Isotónico y energético', 'Refresco de cola', 'Refresco de naranja y de limón', 'Tónica y bitter', 'Refresco de té y sin gas']]


In [264]:
#root > div.grid-layout > div.grid-layout__sidebar > ul > li.category-menu__item.open > div > ul


selection = driver.find_elements(By.CSS_SELECTOR,"#root > div.grid-layout > div.grid-layout__sidebar > ul > li.category-menu__item.open > div > ul")
sub_button = driver.find_elements(By.CSS_SELECTOR, "#root > div.grid-layout > div.grid-layout__sidebar > ul > li.category-menu__item.open > div > ul > li.subhead1-r.category-item.category-item--selected > button")

print(sub_button[0].text)
sub_button[1].click()

selection[0].text.split('\n')




Aceite, vinagre y sal


['Aceite, vinagre y sal',
 'Especias',
 'Mayonesa, ketchup y mostaza',
 'Otras salsas']

In [301]:

category_opened_array = driver.find_elements(By.CSS_SELECTOR,"#root > div.grid-layout > div.grid-layout__sidebar > ul > li.category-menu__item.open > div > ul")

category_opened_array_length = len(category_opened_array[0].text.split("\n"))

number_of_sub_headers = range(1,category_opened_array_length+1)


# Recorre todos los botones dentro de las categorías.
sub_header_button = driver.find_elements(By.CSS_SELECTOR,"#root > div.grid-layout > div.grid-layout__sidebar > ul > li.category-menu__item.open > div > ul > li:nth-child(1) > button")

for i in number_of_sub_headers:

    sub_header_button = driver.find_elements(By.CSS_SELECTOR,f"#root > div.grid-layout > div.grid-layout__sidebar > ul > li.category-menu__item.open > div > ul > li:nth-child({i}) > button")
    sub_header_button[0].click()
    time.sleep(1)


1
2
3
4
5


In [ ]:
# Vamos a intentar recorrer en categoría Aceite, especias y salsas todas las sub-categorías
categoria_abierta = driver.find_elements(By.CLASS_NAME,"category-menu__item.open")

sub_categorias = []

for categoria in categoria_abierta:
    li = driver.find_element(By.TAG_NAME, "li")
    sub_categorias.append(li.text)
    print(li.text)



Aceite, especias y salsas
Aceite, vinagre y sal
Especias
Mayonesa, ketchup y mostaza
Otras salsas


list